<a href="https://colab.research.google.com/github/aiooq/Files/blob/master/logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ООП дробление (Program, Logger, Event, File) на основе семантических связей.

Семантические связи:
+ Program -> использует -> Logger
++ Logger -> содержит -> Event
++ Logger -> использует -> File
		
Такой подход позволяет: 
+ Логировать исключения возникающие внутри Program;
+ Запоминать целиком последнее событие;
+ Гибкое и быстрое расширение возможностей;
> (например можно сделать функцию парсинга события из файла внутри класса Event и др.)
+ Исключено дублирование кода;
+ Повышается читаемость.

Имена функций классов в новом виде содержат функциональный смысл (на мой взгляд) и изящно выглядят в коде.

Благодаря имени файла в таком формате: 2022.08.31.log
1. Файлы в каталоге удобно сортировать по году, месяцу и дню
2. Расширение используется по назначению
3. Легко делать поиск *.log

In [53]:
import os, fnmatch, traceback
from datetime import datetime

In [54]:
class File():
    def __init__(self, file, encoding='UTF-8'):
        self.file = file
        self.encoding = encoding

    def __open(self, mode):
        return open(self.file, mode, encoding = self.encoding)

    def writeline(self, text):
        with self.__open('a') as f:
            f.write(text + '\n')

    def readlines(self):
        with self.__open('r') as f:
            return f.readlines()

    def clear(self):
        with open(self.file, "w") as f: 
            f.close()

In [42]:
class Event():
    def __init__(self, mode = 'info', text = 'init'):
        self.mode = mode
        self.text = text
        self.time = datetime.now()

    def __call__(self) -> str:
        return f"[{self.time.strftime('%H:%M:%S')}] \t{self.mode} \t{self.text}"

In [46]:
class Logger(object):
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Logger, cls).__new__(cls)
        return cls.instance

    def __init__(self, path='.'):
        self.path = path
        Logger.__create_path(path)
        self.write('info', 'init')

    def __call__(self):
        for item in self.get_current_events():
            print(item, sep='', end='')

    # не используется
    @staticmethod
    def __get_time():
        return datetime.now()

    @staticmethod
    def __create_path(path):
        try:
            os.stat(path)
        except FileNotFoundError:
            os.mkdir(path)

    def __get_path(self, time = datetime.now(), prefix = None):
        return f"{self.path}/{time.strftime('%Y.%m.%d')}.log"

    def write(self, mode, text): # вместо write_log
        self.event = Event(mode, text)
        File(self.__get_path(self.event.time)).writeline(self.event())

    def clear(self): # вместо clear_log
        File(self.__get_path()).clear()   

    def get_current_events(self): # вместо get_logs
        return File(self.__get_path()).readlines()

    def get_last_event(self):
        return self.event()

    def get_filenames(self): # вместо get_all_logs
        files = os.listdir(self.path)
        log_files = [file for file in files 
                     if fnmatch.fnmatch(file, "????.??.??.log")]
        return log_files

In [57]:
class Loggers(Logger):
    pass

In [50]:
class Program():
    def __init__(self):
        self.log = Logger()
        self.event_to_logs = Logger('./logs')

    def __call__(self):
        self.log.write('info','start')
        self.log.write('info','end')

        print('Выводим список лога')
        print(self.log.get_current_events())

        print('Печать лога')
        self.log()

        print('Чистим лог')
        self.log.clear()

        print('Выводим последнее событие')
        print(self.log.get_last_event())

        print('Выводим список файлов лога')
        print(self.log.get_filenames())
        print('Проверка шаблона Singleton')
        print("Object created [log]", self.log)
        print(self.log.path)

        print("Object created [event_to_logs]", self.event_to_logs)
        print(self.event_to_logs.path)

    def deinit(self, mode = 'info', text = 'exit'):
        self.log.write(mode,text)

In [ ]:
if __name__ == '__main__':
    prog = Program()
    try:
        prog();
        #raise 'err'
    except Exception as err:
        prog.deinit('error',traceback.format_exc())
    finally:
        prog.deinit()
        del prog